# Exercise - Autoencoder

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. We will build an autoencoder to learn the representation of good loans. That way, we can see if an unknown loan will be a good or bad loan based on its reconstruction error.  

Note: in the data set, we don't have a column that indicates whether a loan is "good" or "bad". So, we can't train a classification model like we did before.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
good = pd.read_csv('good loans.csv')

unknown = pd.read_csv('unknown loans.csv')

In [3]:
# Note that there is no GOOD/BAD classification. Though, we know that
# these loans are good

good.head()

,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076
3,25300,62540.0,101165.0,DebtCon,ProfExe,0.0,0.0,0.0,195.451331,0.0,25.0,35.200865
4,27700,73148.0,101462.0,DebtCon,ProfExe,10.0,0.0,0.0,264.605389,0.0,33.0,40.475793


In [4]:
good.shape

(1489, 12)

In [5]:
unknown.shape

(1011, 12)

# Data Prep

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

##  Identify the numeric, binary, and categorical columns

In [7]:
# Identify the numerical columns
numeric_columns = good.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = good.select_dtypes('object').columns.to_list()

In [8]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [9]:
categorical_columns

['REASON', 'JOB']

# Pipeline

In [10]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [11]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [12]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for GOOD

In [13]:
#Fit and transform the train data
good_x = preprocessor.fit_transform(good)

good_x

array([[ 0.6365782 , -0.29793911, -0.12177109, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.6546228 ,  1.02893596,  1.62034212, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.18546313, -0.25851565, -0.25756212, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 4.18234262, -0.04383447,  0.81444926, ...,  0.        ,
         0.        ,  0.        ],
       [-1.30321659, -0.47828045, -0.73681873, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03208401, -0.46521555, -0.38146895, ...,  0.        ,
         0.        ,  0.        ]])

In [14]:
good_x.shape

(1489, 20)

# Tranform: transform() for UNKNOWN

In [15]:
# Transform the test data
unknown_x = preprocessor.transform(unknown)

unknown_x

array([[ 2.81095281,  3.755331  ,  3.98956663, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.32079765, -0.54180025, -0.65167516, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.19448543,  0.59006724,  0.48213012, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.79796772,  0.05628738,  0.05541527, ...,  0.        ,
         0.        ,  0.        ],
       [-0.16640662, -0.510536  , -0.45504735, ...,  0.        ,
         0.        ,  0.        ],
       [-0.81601232, -0.92230904, -0.93360606, ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
unknown_x.shape

(1011, 20)

# Build an Autoencoder to learn the representation of GOOD loans

In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
model = keras.models.Sequential()

#Encoder
model.add(keras.layers.InputLayer(input_shape=20))
model.add(keras.layers.Dense(25, activation='relu'))
model.add(keras.layers.Dense(26, activation='relu'))

#Decoder
model.add(keras.layers.Dense(26, activation='relu'))
model.add(keras.layers.Dense(20, activation=None))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                525       
                                                                 
 dense_1 (Dense)             (None, 26)                676       
                                                                 
 dense_2 (Dense)             (None, 26)                702       
                                                                 
 dense_3 (Dense)             (None, 20)                540       
                                                                 
Total params: 2,443
Trainable params: 2,443
Non-trainable params: 0
_________________________________________________________________


In [19]:
adam = keras.optimizers.Adam(learning_rate=0.001)


model.compile(loss='mse', optimizer='Nadam', metrics=['mean_squared_error'])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [21]:

model.fit(good_x, good_x, 
          validation_data = (good_x, good_x),
          epochs=100, batch_size=100, callbacks=callback)

Epoch 1/100
15/15 [==============================] - 1s 9ms/step - loss: 0.6947 - mean_squared_error: 0.6947 - val_loss: 0.6208 - val_mean_squared_error: 0.6208
Epoch 2/100
15/15 [==============================] - 0s 3ms/step - loss: 0.5824 - mean_squared_error: 0.5824 - val_loss: 0.5451 - val_mean_squared_error: 0.5451
Epoch 3/100
15/15 [==============================] - 0s 3ms/step - loss: 0.5231 - mean_squared_error: 0.5231 - val_loss: 0.4988 - val_mean_squared_error: 0.4988
Epoch 4/100
15/15 [==============================] - 0s 4ms/step - loss: 0.4794 - mean_squared_error: 0.4794 - val_loss: 0.4557 - val_mean_squared_error: 0.4557
Epoch 5/100
15/15 [==============================] - 0s 6ms/step - loss: 0.4349 - mean_squared_error: 0.4349 - val_loss: 0.4097 - val_mean_squared_error: 0.4097
Epoch 6/100
15/15 [==============================] - 0s 3ms/step - loss: 0.3881 - mean_squared_error: 0.3881 - val_loss: 0.3622 - val_mean_squared_error: 0.3622
Epoch 7/100
15/15 [===============

15/15 [==============================] - 0s 3ms/step - loss: 0.0106 - mean_squared_error: 0.0106 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 52/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0100 - mean_squared_error: 0.0100 - val_loss: 0.0097 - val_mean_squared_error: 0.0097
Epoch 53/100
15/15 [==============================] - 0s 4ms/step - loss: 0.0095 - mean_squared_error: 0.0095 - val_loss: 0.0091 - val_mean_squared_error: 0.0091
Epoch 54/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0086 - val_mean_squared_error: 0.0086
Epoch 55/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.0081 - val_mean_squared_error: 0.0081
Epoch 56/100
15/15 [==============================] - 0s 4ms/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0076 - val_mean_squared_error: 0.0076
Epoch 57/100
15/15 [=====================

### Check the average MSE on the "good" loans

In [22]:
model.evaluate(good_x, good_x)

47/47 [==============================] - 0s 1ms/step - loss: 0.0016 - mean_squared_error: 0.0016


[0.0015590492403134704, 0.0015590492403134704]

In [23]:
model.evaluate(good_x, good_x)[0]*100

47/47 [==============================] - 0s 959us/step - loss: 0.0016 - mean_squared_error: 0.0016


0.15590492403134704

### Check the average MSE on the "unknown" data

In [24]:
model.evaluate(unknown_x, unknown_x)

32/32 [==============================] - 0s 1ms/step - loss: 0.0382 - mean_squared_error: 0.0382


[0.038192734122276306, 0.038192734122276306]

In [25]:
model.evaluate(unknown_x, unknown_x)[0]*100

32/32 [==============================] - 0s 830us/step - loss: 0.0382 - mean_squared_error: 0.0382


3.8192734122276306

### Do you think the "unknown" loans look like good loans or not (justify your answer using the interpretation of the average MSE values)

In [ ]:
#No, the unknown loans do not look like good loans as the mse value of unkown loans i smuch higher as compared to good loans.